# 再探CNN卷积核

如果学过信号系统或者傅里叶变换等基础数学的话，一定听说过卷积运算，基本概念可以参考[如何通俗易懂地解释卷积？](https://www.zhihu.com/question/22298352)。

简而言之，卷积运算可以理解为：有两个序列，其中一个翻转后再和另一个平移做点积，得到一个新的序列的过程。

那么CNN卷积核执行的运算是不是这样的呢？

首先，看一看原始的卷积运算。

In [4]:
import numpy as np

In [5]:
a1 = np.array([0.3, 0.2, 0.5])
a2 = np.array([5, 3, 2])
np.convolve(a1, a2)

array([1.5, 1.9, 3.7, 1.9, 1. ])

PyTorch中没有直接的convovle运算。有的是神经网络中的卷积运算。

functional中有[conv1d函数](https://pytorch.org/docs/stable/generated/torch.nn.functional.conv1d.html#torch.nn.functional.conv1d)，

先简单补充一点CNN中的卷积计算输出尺寸计算的公式（参考[官方文档](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#torch.nn.Conv1d)）。

$$N=\lfloor{\frac{W-F+2P}{S}+1}\rfloor$$

其中，N为输出尺寸，W是输入的，F是卷积核大小，P是padding填充的大小，S是步长stride。

比如输入大小是3，F卷积核也是3，没有P，S为1，那么输出大小就为1；如果我们想在输入和卷积核大小不变的情况下，加大输出尺寸，那么需要padding。令P=2，可以看到输出就是5了。

对于二维是一样的，只不过宽和高各一个计算公式。

尺寸是针对一个feature（即channel）的“图像”大小的，对于一个[batch, feature, sequence]的张量，一维卷积是合适的；如果输入输出的feature都是1，那么卷积核就是 1\*1\*kernel_width  ($\text{out_channels} , \text{in_channels} , kW)$的大小了。

也就是说，一维卷积输入张量各维度分别是 batch-channel_in-width。卷积核是参数weight（bias默认为None），各维度为 channel-channel_in-width （更多内容参考[文档](https://pytorch.org/docs/stable/generated/torch.nn.functional.conv1d.html#torch.nn.functional.conv1d)）

In [6]:
import torch
from torch.nn import functional as F

In [7]:
inputs = torch.Tensor(np.array([[[5, 3, 2]]]))
filters = torch.Tensor(np.array([[[0.3, 0.2, 0.5]]]))
F.conv1d(inputs, filters)

tensor([[[3.1000]]])

可以看到，卷积计算就是直接进行的乘积求和。下面我们手动加上padding和翻转，看看结果。

In [12]:
inputs = torch.Tensor(np.array([[[0, 0, 5, 3, 2, 0, 0]]]))
filters = torch.Tensor(np.array([[[0.5, 0.2, 0.3]]]))
F.conv1d(inputs, filters)

tensor([[[1.5000, 1.9000, 3.7000, 1.9000, 1.0000]]])

可以看到，加入padding和翻转filter之后可以达到同样的效果，所以这也是为什么虽然没有完全和convolve的定义一样但是仍然称作卷积的原因。本质上仍是一样的。

In [15]:
inputs = torch.Tensor(np.array([[[5, 3, 2]]]))
filters = torch.Tensor(np.array([[[0.5, 0.2, 0.3]]]))
F.conv1d(inputs, filters, padding=2)

tensor([[[1.5000, 1.9000, 3.7000, 1.9000, 1.0000]]])

在水文计算中，也经常遇到卷积运算，比如单位线，瞬时单位线可以看作是单位冲激函数在线性时变系统上的响应。不过也有的模型虽然采用单位线的说法，但是更倾向于指明时段产流在后续给时段上的分配情况，比如GR4J模型中的unit hydrograph。本质上来说，两者应该是一致的，都是强调单位时段水量在后续各时段上的重新分布。这里我们以后一种为例，这时候各时段坐标值之和为1（水量守恒）:

In [16]:
def s_curves1(t, x4):
    """
        Unit hydrograph ordinates for UH1 derived from S-curves.
    """

    if t <= 0:
        return 0
    elif t < x4:
        return (t / x4) ** 2.5
    else:  # t >= x4
        return 1

In [21]:
import math
X4=3
nUH1 = int(math.ceil(X4))
uh1_ordinates = [0] * nUH1
for t in range(1, nUH1 + 1):
    uh1_ordinates[t - 1] = s_curves1(t, X4) - s_curves1(t - 1, X4)
    
uh1_ordinates

[0.06415002990995841, 0.29873733939125313, 0.6371126306987884]

如果我们想要让卷积核保持和为1的约束，我们可以考虑使用softmax函数，将输入softmax的参数作为优化的对象，生成的变量作为卷积核，可以继续执行卷积运算。

先简单回顾下softmax函数，默认的softmax是对每列数据进行运算的，保证每列的数据之和为1，可以通过指定dim来对行运算，如下所示：

In [24]:
from torch import nn
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output

tensor([[0.1509, 0.3886, 0.4605],
        [0.6525, 0.1027, 0.2448]])

functional的softmax函数可以达到一样的效果。看一个三维的tensor的例子：

In [32]:
input_3d = torch.randn(1,1,3)
F.softmax(input_3d, dim=2)

tensor([[[0.3510, 0.2587, 0.3903]]])

下面是简单的指定卷积核形式之后的卷积运算（实际应用还需要注意权重参数初始化等），更多实践方法可以参考[这里](https://www.programmersought.com/article/10583895621/)：

In [34]:
class KernelConv(nn.Module):
    def __init__(self):
        super(KernelConv, self).__init__()        

    def forward(self, input, kernel):
        # the dim of input: batch-channel_in-width
        # the dim of kernel: channel-channel_in/groups-width
        uh = F.softmax(kernel, dim=2)
        output = F.conv1d(input, uh,padding=kernel.shape[-1]-1)
        return output

In [36]:
kConv = KernelConv()
inputs = torch.Tensor(np.array([[[5, 3, 2]]]))
# 随便给定一组数据作为filter的前序输入
filters = torch.Tensor(np.array([[[1, 3, 5]]]))
kConv(inputs, filters)

tensor([[[4.3341, 3.1870, 2.1649, 0.2822, 0.0318]]])

知道以上内容可以帮助我们更进一步地了解汇流计算，并对汇流计算如何与神经网络联系起来有更多的直观认识。